In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [2]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [3]:
bs=52 # 52 - Jeremey, 20 - default

In [4]:
bptt= 70 #70 - Jeremey, 35 - default

In [5]:
def create_data_lm(tokens,bs):
    import itertools
    # Collapse into a single large array
    tokens=np.asarray(list (itertools.chain(*tokens)))
    # How many batches
    n_batch=len(tokens)//bs
    # Truncate to exclude the ones at the end
    tokens=tokens[:bs*n_batch]
    # Reshape
    tokens=tokens.reshape(bs,-1)
    return tokens

In [6]:
train_tokens=create_data_lm(df_train['tokens'],bs)
valid_tokens=create_data_lm(df_valid['tokens'],bs)
test_tokens=create_data_lm(df_test['tokens'],bs)
train_tokens.shape, valid_tokens.shape, test_tokens.shape

((52, 396650), (52, 43895), (52, 143259))

In [7]:
def get_batch(source, i):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [8]:
n_batch=train_tokens.shape[1]
for i in range(0,n_batch,bptt):
    seq_len=min(bptt,n_batch-1-i)
    x=train_tokens[:,i:i+seq_len]
    y=train_tokens[:,i+1:i+1+seq_len]
    print (x.shape, y.shape)
    break

(52, 70) (52, 70)


In [9]:
x.shape,y.shape

((52, 70), (52, 70))

In [10]:
x,y

(array([[   41,    42,    43,    40, ...,     2,  1896,   463,     4],
        [ 1071,    23,    67,    39, ...,     2,    98,    11,    63],
        [   54,    34,     7,   190, ...,     2,    87,    34,  5998],
        [    7,  1213,     8,   453, ...,   406,     4,     2,    46],
        ...,
        [   60,   126,   931,     6, ...,   114,    39,   599,    63],
        [    2, 15094,   132,    65, ...,    56,  1350,   209,   281],
        [11783,   658,   318,     3, ...,     6,    64,    79,  1268],
        [   42,    43,    40,     2, ...,   708,   111,     4,     2]]),
 array([[   42,    43,    40,    13, ...,  1896,   463,     4,     2],
        [   23,    67,    39,    18, ...,    98,    11,    63,    28],
        [   34,     7,   190,  1732, ...,    87,    34,  5998,    47],
        [ 1213,     8,   453,    21, ...,     4,     2,    46,     7],
        ...,
        [  126,   931,     6,   126, ...,    39,   599,    63,   314],
        [15094,   132,    65,   411, ...,  1350, 

In [11]:
x.shape,y.shape

((52, 70), (52, 70))

In [12]:
train_tokens.shape

(52, 396650)

## pre-trained GLOVE

In [13]:
PATH_GLOVE='/home/kirana/Documents/phd/data/pre-trained/glove/glove.6B'

In [14]:
ls {PATH_GLOVE}

glove_50_glove_100.tgz  glove.6B.300d.txt  wordvectors_glove50
glove.6B.100d.txt       glove.6B.50d.txt
glove.6B.200d.txt       word2idx_glove50


In [15]:
def load_glove_vectors(path=f'{PATH_GLOVE}/glove.6B.50d.txt'):
    f=open(f'{path}','r')
    glovelines=f.readlines()
    word2idx={}
    idx=0
    wordvectors={}
    for line in glovelines:
        line=line.split(' ')
        vector=np.asarray(line[1:]).astype(np.float32)
        word2idx[line[0]]=idx
        idx=idx+1
        wordvectors[line[0]]=vector
    return word2idx,wordvectors

In [19]:
word2idx_glove,wordvectors_glove=load_glove_vectors(f'{PATH_GLOVE}/glove.6B.200d.txt')
import pickle
pickle.dump(word2idx_glove,open(f'{PATH_GLOVE}/word2idx_glove200','wb'))
pickle.dump(wordvectors_glove,open(f'{PATH_GLOVE}/wordvectors_glove200','wb'))

In [19]:
word2idx_glove=pickle.load(open(f'{PATH_GLOVE}/word2idx_glove200','rb'))
wordvectors_glove=pickle.load(open(f'{PATH_GLOVE}/wordvectors_glove200','rb'))

In [24]:
len(word2idx_glove)

400000

In [25]:
[*word2idx_glove.keys()][:5]

['the', ',', '.', 'of', 'to']

In [26]:
wordvectors_glove['the'].shape

(200,)

In [27]:
glove_mtx=np.array([wordvectors_glove[key] for key in [*word2idx_glove.keys()]])

In [28]:
glove_mtx.shape

(400000, 200)

In [29]:
# In our list but not in Glove
len(set(itos).difference(set([*word2idx_glove.keys()])))

5072

In [30]:
len(set(itos).difference(set([*word2idx_glove.keys()])))/len(itos)

0.0845728006402988

In [31]:
glove_mtx=np.zeros((len(itos),glove_mtx.shape[1]))

In [32]:
wordvectors_glove['the']

array([-0.071549,  0.093459,  0.023738, -0.090339, ..., -0.008877,  0.33617 ,  0.030591,  0.25577 ], dtype=float32)

In [33]:
enc_wgts = glove_mtx # converts np.ndarray from torch.FloatTensor.output shape: (238462, 400)
row_m = enc_wgts.mean(0) # returns the average of the array elements along axis 0. output shape: (400,)

In [34]:
for i,word in enumerate(itos):
    try:
        glove_mtx[i]=wordvectors_glove[word]
    except:
        glove_mtx[i]=row_m

In [35]:
glove_mtx[3]

array([-0.071549,  0.093459,  0.023738, -0.090339, ..., -0.008877,  0.33617 ,  0.030591,  0.25577 ])

In [36]:
enc_wgts.shape, len(itos)

((59972, 200), 59972)

In [37]:
n_emb=enc_wgts.shape[1]

In [38]:
new_w=enc_wgts

In [39]:
new_w.shape

(59972, 200)

## Model Architecture

In [40]:
from torch import nn
import torch
from adaptive import *
from torch.autograd import Variable

In [41]:
n_inp=len(itos)
n_emb=200 #650
n_hidden=200 #650
n_layers=2
dropout=0.5
wd=1e-7

In [42]:
class language_model (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,adaptive_log_softmax=True,tie_weights=True):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx
        self.adaptive_log_softmax,self.tie_weights=adaptive_log_softmax,tie_weights
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        self.gen_hidden()
        self.create_architecture()
        if pretrain_mtx is not None:
            print ("initializing")
            self.initialize_glove()
            
        if self.adaptive_log_softmax is False:
            self.criterion=nn.CrossEntropyLoss()
        
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer
        self.encoder=nn.Embedding(self.n_inp,self.n_emb)
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=False)
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        if self.adaptive_log_softmax:
            # Adaptive Log Softmax Loss
            self.adaptive_softmax=AdaptiveLogSoftmaxWithLoss(self.n_hidden,
                                    self.n_inp,
                                    cutoffs=[round(self.n_inp/15),3*round(self.n_inp/15)],
                                    div_value=4,
                                    get_full_prob=True)
        else:
            self.decoder=nn.Linear(self.n_hidden,self.n_inp)
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
        if self.tie_weights:
            self.decoder.weight.requires_grad=False
        
    
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True
        if self.tie_weights:
            self.decoder.weight.requires_grad=True
        
    def initialize_glove(self):
        self.encoder.weight.data=torch.Tensor(self.pretrain_mtx)
        if self.tie_weights:
            self.decoder.weight=self.encoder.weight
    
    def gen_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb):
        #import pdb
        #pdb.set_trace()
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        out,new_hidden=self.lstm(embs,self.hidden)
        out=self.dropout_op(out)
         # Wrap the hidden state in a new tensor without the gradients
        self.hidden=(Variable(new_hidden[0].data,requires_grad=False).to(self.device),\
                     Variable(new_hidden[1].data,requires_grad=False).to(self.device))
        if self.adaptive_log_softmax:
            out=out.reshape(out.size(0)*out.size(1),out.size(2))        # output is of shape n_batch * n_seq * n_hidden
      
            out=self.adaptive_softmax(out,Yb.view(-1))
            loss=out.loss
            preds=out.output_full
        else:
            #import pdb
            #pdb.set_trace()
            preds=self.decoder(out.contiguous().view(out.size(0)*out.size(1), out.size(2)))
            loss=self.criterion(preds,Yb.contiguous().view(-1))

        return preds, loss  

In [43]:
def accuracy_multinomial(preds,actual):
    preds=preds.max(1)[1]
    correct=preds==actual
    return correct.float().sum()/len(correct)

In [44]:
device="cuda:1"
model=language_model(n_inp,n_emb,n_hidden,n_layers,False,bs,device,0.05,0.5,0.5,new_w,False,True)


initializing


In [45]:
new_w.shape

(59972, 200)

In [46]:
model.encoder.weight.data.shape,model.decoder.weight.data.shape

(torch.Size([59972, 200]), torch.Size([59972, 200]))

In [47]:
nn.Linear(n_hidden,n_inp).weight.data.shape, n_hidden, n_inp

(torch.Size([59972, 200]), 200, 59972)

### Test if model forward works

In [48]:
model.decoder.weight.shape,model.encoder.weight.shape

(torch.Size([59972, 200]), torch.Size([59972, 200]))

In [49]:
x.shape

(52, 70)

In [50]:
if 1==0:
    model.forward(torch.LongTensor(x),torch.LongTensor(y))

In [51]:

model=model.to(device)

## Learner

In [52]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
  
        
    
    def fit (self,Xb,Yb,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb)
        
       
            
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1))
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.25)
            if 1==0:
                for p in self.model.parameters():
                    p.data.add_(self.lr, p.grad.data)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        n_batch=iterator.shape[1]
        epoch_loss,epoch_acc,i=0,0,0
        self.model.gen_hidden()
        for k,i in enumerate(range(0,n_batch,self.bptt)):
            seq_len=min(bptt,n_batch-1-i)
            Xb=train_tokens[:,i:i+seq_len]
            Yb=train_tokens[:,i+1:i+1+seq_len]
            Xb=torch.LongTensor(Xb)
            Yb=torch.LongTensor(Yb)
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/k
        epoch_acc=epoch_acc/k
        
        if 1==0:
            lr /= 4.0
            # Freeze all the layers initially
            for param in resnet.parameters():
                param.requires_grad=False
            torch.save(resnet,'resnet')
            torch.save(resnet.state_dict(),'resnet_state_dict')
            resnet.load_state_dict(torch.load('resnet_state_dict'))
            for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr  
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [53]:
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_multinomial

In [54]:
n_batch=np.int(np.ceil(train_tokens.shape[1]/bptt))
n_batch

5667

In [55]:
model.freeze_embedding()

In [56]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(False, False)

In [57]:
learner=Learner(model,optimizer,accuracy_multinomial,device,bptt,500,0.25)

In [58]:
model.n_inp

59972

In [59]:
learner.run_epochs(train_tokens,valid_tokens,3)

Batch:500 7.8526061296463014  0.048273628686089066
Batch:1000 7.568344240665436  0.0656211562550161
Batch:1500 7.385362540880839  0.07556172394395495
Batch:2000 7.243246864318848  0.08274395888776052
Batch:2500 7.12381884689331  0.08824395966613666
Batch:3000 7.020513757864634  0.09280357567539128
Batch:3500 6.928043052945818  0.09667009880985798
Batch:4000 6.844364072918892  0.10002301305398578
Batch:4500 6.768656057569716  0.10318645177000306
Batch:5000 6.699413502311707  0.10720423592370935
Batch:5500 6.634127405773509  0.11119016533315351
Epoch:0 Loss:6.6136987492089725
Batch:500 5.621094839096069  0.17891704100370406
Epoch:0 Loss:6.6136987492089725 Accuracy:0.11241941579562167 Loss:5.619333422355104 Accuracy:0.17869302549620755
Batch:500 5.8941029958724975  0.15902033844590188
Batch:1000 5.858174221038818  0.16109726160764695
Batch:1500 5.829671250979105  0.16322107116381326
Batch:2000 5.802444364070892  0.16539520118385553
Batch:2500 5.778238586616516  0.16717209668159486
Batch:3

In [60]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 5.53926357460022  0.18324890884757042
Batch:1000 5.534780735969544  0.18300989787280558
Batch:1500 5.535018832524617  0.1827939638396104
Batch:2000 5.532855356216431  0.18289176600426435
Batch:2500 5.530926789855957  0.18288506276011468
Batch:3000 5.529588640054067  0.18288892726103465
Batch:3500 5.527603641646249  0.183052362625088
Batch:4000 5.526070582270623  0.18319210946559905
Batch:4500 5.524496164957682  0.18330592966079712
Batch:5000 5.523850279712677  0.18339110670089723
Batch:5500 5.5221725163026285  0.18357428349419075
Epoch:0 Loss:5.521768107500086
Batch:500 5.22363712978363  0.21065495431423187
Epoch:0 Loss:5.521768107500086 Accuracy:0.1836117340710068 Loss:5.222834784828685 Accuracy:0.21049149149057397


In [62]:
torch.save(model.state_dict(),f'{PATH}/inter/glove200_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/glove200_learner_state_dict')

In [63]:
model.unfreeze_embedding()

In [64]:
model.encoder.weight.requires_grad, model.decoder.weight.requires_grad

(True, True)

In [65]:
learner.run_epochs(train_tokens,valid_tokens,3)

Batch:500 5.312142787933349  0.20092143654823302
Batch:1000 5.220643448829651  0.20611649253964423
Batch:1500 5.167726959228515  0.208644331852595
Batch:2000 5.126642462968826  0.210777894616127
Batch:2500 5.093956485748291  0.21232418619990348
Batch:3000 5.067064871470134  0.21367510228852432
Batch:3500 5.043712053571428  0.21492316636017392
Batch:4000 5.023481311321259  0.21595509352535008
Batch:4500 5.005813073158264  0.21689677559667164
Batch:5000 4.990340952396393  0.2177352861136198
Batch:5500 4.9752811776074495  0.21863832317699086
Epoch:0 Loss:4.970837374689551
Batch:500 4.539077677726746  0.2520000110864639
Epoch:0 Loss:4.970837374689551 Accuracy:0.21889011698889724 Loss:4.538255211649139 Accuracy:0.2518512300041874
Batch:500 4.814705096244812  0.22838078156113625
Batch:1000 4.804414977550507  0.22867336396872998
Batch:1500 4.8006322940190636  0.228641953676939
Batch:2000 4.794489390134811  0.22892899580299855
Batch:2500 4.788668313407898  0.22904748484492302
Batch:3000 4.7837

In [66]:
torch.save(model.state_dict(),f'{PATH}/inter/glove200_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/glove200_learner_state_dict')

In [67]:
learner.run_epochs(train_tokens,valid_tokens,2)

Batch:500 4.659603902816772  0.23899616533517837
Batch:1000 4.6538007450103756  0.23880990178883077
Batch:1500 4.653609954516093  0.23856576255957285
Batch:2000 4.651165057182312  0.2385523467734456
Batch:2500 4.648830134391785  0.2384993522465229
Batch:3000 4.646925424098969  0.2385403046409289
Batch:3500 4.644628941808428  0.23870409328171185
Batch:4000 4.642290043950081  0.23883792372420432
Batch:4500 4.640284406873915  0.23893133036295572
Batch:5000 4.638935650253296  0.23903336329758168
Batch:5500 4.636414554595947  0.23926025142723864
Epoch:0 Loss:4.635876988927004
Batch:500 4.328870507240295  0.26718572479486463
Epoch:0 Loss:4.635876988927004 Accuracy:0.23929453880236504 Loss:4.328198654229561 Accuracy:0.26693066759733114
Batch:500 4.621987651824951  0.24143572640419006
Batch:1000 4.6171903052330014  0.24126347355544567
Batch:1500 4.617815558115641  0.24099744781851767
Batch:2000 4.616146491289139  0.24096470968425274
Batch:2500 4.614309655380249  0.24084330854415895
Batch:3000 

In [68]:
np.exp(3.9), np.exp(4.28)

(49.40244910553017, 72.24044000732538)

In [69]:
torch.save(model.state_dict(),f'{PATH}/inter/glove200_model_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/glove200_learner_state_dict')

In [80]:
if 1==0:
    learner.lr_find(1e-4,1e-1,valid_tokens,n_batch)
    learner.plot_lrs(50)
    len(learner.lrs)
    len(learner.losses)

Batch:500 7.944759424209595  0.03532285663112998
Batch:1000 7.877773783683777  0.03720142808556557
Batch:1500 7.802010853131613  0.03968857092410326
Batch:2000 7.722838351964951  0.04160571377538145
Batch:2500 7.658665308570862  0.043098285208642485


## Transfer Learning to predict RNN